# Practice using the Lesson 1 notebook
> Notes diving deeper into the Lesson 1 notebook from V4 of the Practical Deep Learning for Coders course

- toc:true
- branch: master
- badges: true
- author: Nissan Dookeran
- categories: [fastai, course_v4]
- show_tags: true
- comments: true

## Notes from Jeremy's Lesson 1 lecture

From [Lesson 1 of fastbook](https://github.com/fastai/fastbook/blob/master/01_intro.ipynb) as the basis for [Lesson 1 of the V4 course](https://github.com/fastai/course-v4/blob/master/nbs/01_intro.ipynb) currently being run we're using copies of the notebook code embedded in their new book. So below are my experiments from what I learnt here, starting from scratch but drawing heavily from their textbook code. I run this locally on Windows 10 (I can see Jeremy's eye rolling if he reads this since he recommended against this for beginners) but in my defense I did do the course last year and had my laptop configured with V1 of Fastai running so thought I'd see where the stress points were, but to keep productive I also run on Google Collab which is honestly much easier. As a caveat, I'm writing this article while running Jupyter Notebook on the Windows 10 environment so if some code looks slightly modified I'll try to mention explicitly this is a Win10 specific change. Now, on to the experimentation.

First if you're running this locally make sure you have the "Trusted" button enable for this notebook. 

Now we're going to make sure fastai is installed into my local environment

In [5]:
pip install fastai2

Note: you may need to restart the kernel to use updated packages.


Now in the notebook and the V4 course repo, Jeremy hides alot of the guts of getting things up and running with

In [ ]:
#hide
from utils import *

In [6]:
from utils import * #this will crash if you run it outside the folder since `utils.py` is in the root folder of the notebooks

ModuleNotFoundError: No module named 'utils'

But since I like to dig into things I'm going to try to parse the `utils.py` file and pull out just the pieces we need to get Lesson 1 stuff up and running

In [7]:
from fastai2.vision.all import *

I also want to test my environment to make sure I have a Nvidia GPU and probe for some details on it

In [8]:
import torch
torch.cuda.device(0)


In [9]:
torch.cuda.get_device_name(0)

'Quadro P2000'

In [10]:
torch.cuda.is_available()


True

In [11]:
torch.backends.cudnn.enabled

True

In [12]:
import torch.cuda
if torch.cuda.is_available():
    print('PyTorch found cuda')
else:
    print('PyTorch could not find cuda')

PyTorch found cuda


There's a block of code in the notebook that executes a learner on images data of both cats and dogs and their breeds. I'm segmenting that code below and putting notes on each

First he gets the data from the built in `URLS` object for `PETS` data which is a simple Amazon S3 storage bucket location containing what looks like either the originally or slightly modified copy of [The Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/)

In [13]:
print(URLs.PETS)

https://s3.amazonaws.com/fast-ai-imageclas/oxford-iiit-pet.tgz


So we use a method `untar_data` to decompress this into our local `images` folder. In the next block if you uncomment the code and press `Shift-Tab` 3 (or 4 times for a pop out window) you can see the documentation on the function and the parameters it takes

In [14]:
untar_data #Press `Enter` to see some info or use `Shift+Tab` 3 or 4 times to see more formatted documentation

<function fastai2.data.external.untar_data(url, fname=None, dest=None, c_key='data', force_download=False, extract_func=<function file_extract at 0x000002B2F4592948>)>

In [15]:
path = untar_data(URLs.PETS)/'images'

So this had me scratching for awhile when I read the notebook but then in the video Jeremy explained the need for this function. Apparently because it is a mixture of both cats and dogs and their breeds, the way to identify data as being for a cat is to Capitalize The First Letter For Any Cat Breed. So the function below would be reused when classifying data as a Cat before going deeper into identifying its breed.

In [16]:
def is_cat(x): return x[0].isupper()

The `ImageDataLoaders` function is handy as it has a variety of extensions to make loading from various image sources much easier. FastAI has this concept of a `DataLoader` that holds the data we're going to train and validate our model on. Because I'm running this on Windows 10, I had to add an additional parameter of `num_workers=0` to the call to make sure it doesn't crash. For Linux users, you can safely remove this call.

In [17]:
ImageDataLoaders.from_ #press Tab once to see all possible calls

AttributeError: type object 'ImageDataLoaders' has no attribute 'from_'

In [18]:
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224), num_workers=0)

So now we're going to call the learner that will train the model to recognise the various cats and dogs breeds in our model using the [ResNet34](https://arxiv.org/abs/1512.03385) pre-trained model. A pre-trained model is a very cool concept. Normally to start training a series of weights are chosen at random and over time are changed to better match a specific set of data. These weights are re-usable since basically it lets us jumpstart training a new, related dataset as these pre-trained series of weights are a better starting point than "random" for the new more niche subset of related data. Here we also specifiy the metric we're using as a measure of accuracy, fastai defines one for us that's useful, appropriately called `error_rate`

In [19]:
error_rate # Press Shift+Tab 3 times to see the documentation on this

<function fastai2.metrics.error_rate(inp, targ, axis=-1)>

In [20]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)

Fast.ai V2 also introduced this nice method called `fine_tune` which basially runs a cycle of training again using a learning rate it has pre-determined as best for training to "fine tune" the model to produce better accuracy. I won't propose to understand it fully yet, but from what I know it freezes a fixed number of the hidden layers of the pre-trained model, and unfreezes the last few layers so their weights can be changed to better match the niche dataset. The logic is sound as the first set of layers would already be really good at recognising general features common in all images of the superset, but the last few layers can now be fine tuned to match the explicit features in the specific classifications of the niche dataset we are presenting for training in this model. As I learn I may write more about how I understand this function to work and append this post to match what I learn. Note, this step might take some time to execute depending on the speed and memory of the GPU you have running. Google Collab would take about 5 minutes. My local laptop running Windows 10 takes a little longer.

In [21]:
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.167801,0.024875,0.007442,07:17


epoch,train_loss,valid_loss,error_rate,time
0,0.052814,0.014244,0.006089,05:19


So now we have a trained model that looks like it has a pretty low error rate and we want to test it with something it hasn't seen before. This library in the next code block was added early in V3 of the course and is more prominent in V4 and lets you put GUI elements you're used to in a normal application (like a file uploader) into the notebook. I did a quick web search for a cat, downloaded that file and used it here to test.

In [19]:
from ipywidgets import widgets
uploader = widgets.FileUpload()
uploader

FileUpload(value={}, description='Upload')

The next block of code simply looks at the image we loaded, passes it to our model for an evaluation of whether or not it thinks it's a cat. I'd recommend having both a dog and a cat image available to test on your own

In [20]:
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

Is this a cat?: True.
Probability it's a cat: 1.000000


And there you go. That simple. You're now a deep learning practitioner like me so celebrate the accomplishment. 
Hopefully you feel less like this:



> youtube: https://youtu.be/ygFoN3hKi20

And more like this:

> youtube: https://youtu.be/535Zy_rf4NU

## Notes from first few minutes Jeremy's Lesson 2 lecture

Jeremy continued to cover the Chapter 1 notebook from Fastbook in Lesson 2. Appending the notes here to keep them naturally aligned to the Fastbook content for when I get my copy (or you yours) to be able to map it all back more easily

First bring back the code we're taking a look at

> Tip: 
- Use the [fastbook book git repository](https://github.com/fastai/fastbook) as well as the [course-v4 repository](https://github.com/fastai/course-v4) for reference. The *fastbook* repo should be used as reading material while the *course-v4* repo can be used to just see the sample code and experiment to ensure you're understanding the material

In [2]:
from fastai2.vision.all import * #since I don't have utils.py here, will just pull the lines from it we need
path = untar_data(URLs.PETS)/'images'
def is_cat(x): return x[0].isupper()

Now, let's take a look at the bit he's exploring for the notes below

In [3]:
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224), num_workers=0) #num_workers=0 is needed for Windows only

- First thing to note, is we have a labelling function `label_func`, in this case `is_cat(x)` which all it does is check the first letter of the file name, because the files are labelled in such a way that an UPPERCASE first letter in the file name indicates it is a cat breed.

- This example code allows us to predict a category, i.e. the label is a category. In this case, our model is a *classification model*
    > If we're trying to predict a variable number, like an age, or location then this is a *regression model*

- What `valid_pct=0.2` does in the sample code given is puts aside 20% of the data in training to test the model to see how accurate the model is. This is called a *validation set* and is useful in detecting *overfitting*
- Overfitting happens when you train the model for too long, or with too little data or with too many parameters, the accuracy of the model starts to go down, since it measures predictions on the validation set and when this value starts to go down, this is a sign of overfitting. 

- Looking at `cnn_learner(dls, resnet34, metrics=error_rate)`. This is a *learner*, which contains your data (`dls`), a function (`resnet34`) which we are optimising for our data, and a list of functions (`metrics=error_rate`) you want executed and printed out after running each *epoch*. It figures out what are the *parameters* that best cause the function `resnet34` to best match the labels in our data in `dls`.
    - learner
    - parameters
    - epoch - when you look at every image in the training set once.
    - `resnet34` is a particular *architecture* [Resnet](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035) that has been shown to be very good at computer vision problems. 34 in `resnet34` indicates how many layers there are. The higher numbered versions (resnet50 etc) take more parameters, use more memory, take longer to train, are more likely to overfit but can also create more complex models
    - architecture


`metrics` is a function that measures the quality of the predictions using our *validation set*.
`error_rate` prints out what percent of the training dataset are being incorrectly classified by the model. `accuracy` is another metric and is calculated as (1 - *error_rate*)

There is potentially a difference between the *metric* and the *loss*, but they are closely related. Attribution to [Arthur Samuel](https://en.wikipedia.org/wiki/Arthur_Samuel)'s insight here. 
- We need a way to measure how well our model is doing so when we change the *parameters* we cna figure out which set of parameters make that performance measurement get better or worse. The *loss* is that performance measurement.

*Error rate* and *accuracy* do not indicate if we are getting better or worse if we change our *parameters* just a little bit, since we may overall be predicting exactly the same number of items correctly, so these are not measures of *loss* since potentially we could be not predicting as well on the data in the validation set once the parameters are changed, but still well enough to give the labelled output value as the prediction.
You generally care most about the metrics, but the program cares about the loss, since this tells it how to update the parameters when running the model against the training data in the next *epoch*.


- We care about how well the model predicts not just with the data we're training with, but with data it has never seen before.
- While we can *cheat* by developing a model that generates great metrics for our validation set, we can set aside a 3rd set of data, a *test set* that isn't used in either training or generating validation metrics for the model, but only used when the entire project is completed.

Jeremy starts going into fine-tuning, transfer learning, and how using AlexNet example what makes sense intuitively is proven.
Parameters vs hyper-parameters. TODO is add notes from here as I go over again this material.